# Glove Model

In [1]:
import numpy as np
import pandas as pd
import gensim
import os

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense,Embedding,GRU,LSTM
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

# Get the data

In [4]:
data=pd.read_excel('cleaned_hm.xlsx')
data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [5]:
data.shape

(100535, 9)

# Basic data cleaning

In [6]:
data['predicted_category'].value_counts()

predicted_category
affection           34168
achievement         33993
enjoy_the_moment    11144
bonding             10727
leisure              7458
nature               1843
exercise             1202
Name: count, dtype: int64

In [7]:
data['num_sentence'].value_counts()

num_sentence
1     83711
2      9542
3      3847
4      1624
5       821
6       336
7       183
8       107
10       68
9        61
11       35
13       26
12       21
16       17
18       17
14       14
17       14
19       12
21       10
15        7
23        7
25        7
26        5
24        5
22        4
30        3
31        3
20        3
29        3
27        2
32        2
40        2
37        2
69        1
60        1
34        1
42        1
44        1
48        1
35        1
46        1
51        1
56        1
53        1
58        1
28        1
45        1
Name: count, dtype: int64

In [8]:
# deleting the happy moments with less than 10 sentences

mod_data=data.loc[data['num_sentence']<=10]
mod_data['predicted_category'].value_counts()

predicted_category
affection           34020
achievement         33966
enjoy_the_moment    11115
bonding             10700
leisure              7458
nature               1839
exercise             1202
Name: count, dtype: int64

In [9]:
# categorical to numerical

encode={
    "affection":0,
    "achievement":1,
    "bonding":2,
    "enjoy_the_moment":3,
    "leisure":4,
    "nature":5,
    "exercise":6
}
encode

{'affection': 0,
 'achievement': 1,
 'bonding': 2,
 'enjoy_the_moment': 3,
 'leisure': 4,
 'nature': 5,
 'exercise': 6}

In [10]:
mod_data['predicted_category'].apply(lambda x:encode[x])
mod_data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


# Data cleaning for NLP

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shabaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
import string
#nltk.download('all')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

happy_lines=list()
lines=mod_data['cleaned_hm'].values.tolist()

for line in lines:
    tokens=word_tokenize(line)
    tokens=[w.lower() for w in tokens]
    
    # remove Punctuations 
    table=str.maketrans('','', string.punctuation)
    stripped=[w.translate(table) for w in tokens]
    
    #remove no alphabetic characters
    words=[word for word in stripped if word.isalpha()]
    happy_lines.append(words)
    
happy_lines[:5]    

TypeError: expected string or bytes-like object, got 'float'